In [ ]:
import os
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = hf_token

In [ ]:
!pip install transformers==4.48.0 peft==0.14.0 trl==0.13.0 bitsandbytes==0.45.3 accelerate==1.2.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fine_tuned_model_path = "/content/drive/MyDrive/fine_tune_output/checkpoint-50"

In [ ]:
# Quantization figuration to 4 bits
from transformers import BitsAndBytesConfig
import torch

quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [ ]:
# Load a basic Llama 3 model
from transformers import AutoModelForCausalLM
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    quantization_config = quantization_config,
    device_map = {"": 0}
)

In [ ]:
# Load my fine-tuned model
fine_tuned_model_cp_50 = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_path,
    quantization_config=quantization_config,
    device_map = {"": 0}
)

In [ ]:
# Tokenizer
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

In [ ]:
# Prompt/Response Format
EOS_TOKEN = tokenizer.eos_token

def convert_to_alpaca_format(instruction, response):
    alpaca_format_str = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\
    \n\n### Instruction:\n{instruction}\n\n### Response:\n{response}\
    """

    return alpaca_format_str

In [ ]:
def test_model(instruction_str, model):
    inputs = tokenizer(
    [
        convert_to_alpaca_format(instruction_str,"",) # Response field is empty because it is an inference stage!
    ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True, temperature = 0.05, top_p = 0.95) # Basic inference function in HF
    return(tokenizer.batch_decode(outputs)[0]) # Decode token IDs into words

In [ ]:
questions = [
    "List three ways to reduce plastic waste in daily life.",
    "Write a haiku about artificial intelligence.",
    "Translate '안녕하세요, 오늘 날씨가 좋네요' into English.",
    "How can I improve my public speaking skills?",
    "Explain what the term LLM means in the field of AI.",
    "What is a famous tall tower in Paris?",
    "What is Fine-Tuning?",
    "Find the least common multiple of 15 and 25.",
    "What were the main causes of World War II?",
    "Briefly explain the process of photosynthesis.",
    "Analyze the personality of the protagonist in Shakespeare's 'Hamlet'.",
    "Explain the global impact of climate change.",
    "What is the difference between a list and a tuple in Python?",
    "Can artificial intelligence have consciousness? Why or why not?",
    "Write a short story about space travel.",
    "Explain the effects of inflation on the economy."
]

In [ ]:
answers_dict = {
  "base_model_answers": [],
  "fine_tuned_model_answers": []
}
for idx, question in enumerate(questions):
    print(f"Processing EXAMPLE {idx}==============")
    base_model_output = test_model(question, base_model)
    answers_dict['base_model_answers'].append(base_model_output)
    fine_tuned_model_output = test_model(question, fine_tuned_model_cp_50)
    answers_dict['fine_tuned_model_answers'].append(fine_tuned_model_output)

In [ ]:
def simple_format(text, width=120):
    return '\n'.join(line[i:i+width] for line in text.split('\n') for i in range(0, len(line), width))


for idx, question in enumerate(questions):
    print(f"EXAMPLE {idx}==============")
    print(f"Question: {question}")

    print("<<Base Model response>>")
    base_model_answer = answers_dict["base_model_answers"][idx].split("### Response:")[1]
    print(simple_format(base_model_answer))
    print("---")
    print("<<Fine Tuning Model response>>")
    fine_tuned_model_answer = answers_dict["fine_tuned_model_answers"][idx].split("### Response:")[1]
    print(simple_format(fine_tuned_model_answer))
    print()